In [1]:
import util

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
descriptions, images = util.make_descriptions_and_images()

In [4]:
labels, classes, class_to_index = util.make_labels_and_classes(descriptions)

In [ ]:
s = pd.Series(list(labels.keys()))
mapping = {key : list(val.values()) for key, val in pd.get_dummies(s).to_dict().items()}

In [ ]:
s

0                       c_kefir
1                   ent_cloacae
2         klebsiella_pneumoniae
3         moraxella_catarrhalis
4         staphylococcus_aureus
5    staphylococcus_epidermidis
dtype: object

crop for classification

In [ ]:
cropper = torchvision.transforms.functional.crop
X, y = [], []
for class_, indexes in labels.items():
  curent_class = class_to_index[class_]
  for index in indexes:
    X.append(np.asarray(cropper(Image.fromarray(images[index]), 0, 0, 512, 512)))
    y.append(curent_class)
#X = np.array(X)
#y = np.array(y)
del images

In [ ]:
mins, diffs = [], []
min_, max_ = np.array(X).min(axis=0), np.array(X).max(axis=0)
diff = max_ - min_
del max_
mins.append(min_.transpose(2, 0, 1).astype(np.uint8))
diffs.append(diff.transpose(2, 0, 1).astype(np.uint8))
min_im = Image.fromarray(min_)
diff_im = Image.fromarray(diff)
for j in range(1, 4):
  mins.append(np.asarray(min_im.rotate(90 * j, expand=True)).transpose(2, 0, 1))
  diffs.append(np.asarray(diff_im.rotate(90 * j, expand=True)).transpose(2, 0, 1))
mins.append(np.asarray(min_im.transpose(Image.FLIP_TOP_BOTTOM)).transpose(2, 0, 1))
diffs.append(np.asarray(diff_im.transpose(Image.FLIP_TOP_BOTTOM)).transpose(2, 0, 1))
mins.append(np.asarray(min_im.transpose(Image.FLIP_LEFT_RIGHT)).transpose(2, 0, 1))
diffs.append(np.asarray(diff_im.transpose(Image.FLIP_LEFT_RIGHT)).transpose(2, 0, 1))
for j in range(6):
  diffs[j] = np.where((diffs[j] == 0), np.inf, diffs[j])
def scale(X, rotate):
  return 2 * (X - mins[rotate]) / diffs[rotate] - 1

In [ ]:
np.save('/content/drive/My Drive/ml_contests/digital_breakthrough/classifier_mins.npy', mins[0])
np.save('/content/drive/My Drive/ml_contests/digital_breakthrough/classifier_diffs.npy', diffs[0])

In [ ]:
#X_train, X_test, y_train, y_test = map(np.ndarray.tolist, train_test_split(X, y, test_size=0.3, stratify=y))

In [ ]:
len_ = len(X)
X_train_rot = [0] * len_
for i in range(len_):
  im = Image.fromarray(np.array(X[i], dtype=np.uint8))
  im2 = im.rotate(90, expand=True)
  X.append(np.asarray(im2))
  im3 = im2.rotate(90, expand=True)
  X.append(np.asarray(im3))
  im4 = im3.rotate(90, expand=True)
  X.append(np.asarray(im4))
  
  X.append(np.asarray(im.transpose(Image.FLIP_TOP_BOTTOM)))
  X.append(np.asarray(im.transpose(Image.FLIP_LEFT_RIGHT)))
  X_train_rot += [1, 2, 3, 4, 5]  
  for j in range(5):
    y.append(y[i])

In [ ]:
#X_train, X_test, y_train, y_test = map(np.array, (X_train, X_test, y_train, y_test))

In [ ]:
X, y = map(np.array, (X, y))
X = X.transpose(0, 3, 1, 2)

In [ ]:
X_train = X_train.transpose(0, 3, 1, 2)
X_test = X_test.transpose(0, 3, 1, 2)

In [ ]:
device = torch.device("cuda:0")

In [ ]:
def make_model():
  return nn.Sequential( # 512, 512
    nn.Conv2d(3, 8, 6, padding=2, stride=2),
    #nn.BatchNorm2d(8),
    #nn.Tanh(),
    nn.ReLU(),

    nn.Conv2d(8, 16, 6, padding=2, stride=2),
    #nn.BatchNorm2d(16),
    #nn.Tanh(),
    nn.ReLU(),

    nn.MaxPool2d(2, stride=2), # 256, 256

    nn.Conv2d(16, 32, 3, padding=1),
    #nn.BatchNorm2d(32),
    #nn.Tanh(),
    nn.ReLU(),

    nn.Conv2d(32, 64, 3, padding=1), 
    #nn.BatchNorm2d(64),
    #nn.Tanh(),
    nn.ReLU(),
    
    nn.MaxPool2d(2, stride=2), # 128

    nn.Conv2d(64, 128, 3, padding=1),
    #nn.BatchNorm2d(128),
    #nn.Tanh(),
    nn.ReLU(),
    
    nn.Conv2d(128, 256, 3, padding=1), 
    #nn.BatchNorm2d(256),
    #nn.Tanh(),
    nn.ReLU(),

    nn.MaxPool2d(2, stride=2), # 64

    nn.Conv2d(256, 128, 3, padding=1), 
    #nn.BatchNorm2d(128),
    #nn.Tanh(),
    nn.ReLU(),

    nn.Conv2d(128, 64, 3, padding=1), 
    #nn.BatchNorm2d(64),
    #nn.Tanh(),
    nn.ReLU(),


    nn.MaxPool2d(2, stride=2), # 32

    nn.Conv2d(64, 32, 3, padding=1), 
    #nn.BatchNorm2d(32),
    #nn.Tanh(),
    nn.ReLU(),

    nn.Conv2d(32, 16, 3, padding=1), 
    #nn.BatchNorm2d(16),
    #nn.Tanh(),
    nn.ReLU(),

    nn.MaxPool2d(2, stride=2), # 16
    
    nn.Conv2d(16, 6, 1),
    nn.AvgPool2d((4, 4)),
    nn.Flatten(), 
    #nn.Softmax(dim=1) # 0.29 - 0.08 score = 87 train, 91 test - no augment; w aug 91/92&0.08/0.09 - 10 epoches but conv already
  ).double().to(device=device)

In [ ]:
X_train.shape, X_test.shape, y_train.shape

((1092, 3, 512, 512), (79, 3, 512, 512), (1092,))

In [ ]:
X.shape, y.shape

((1566, 3, 512, 512), (1566,))

In [ ]:
class dataset(data.Dataset):
  def __init__(self, x, y, rot):
    self.x = x
    self.len = x.shape[0]
    self.y = y
    self.rotation = rot

  def __len__(self):
    return self.len

  def __getitem__(self, id):
    return (scale(self.x[id], self.rotation[id]), self.y[id])

In [ ]:
train_dataset = dataset(X, y, X_train_rot)
test_dataset = dataset(X_test, y_test, np.zeros(X_test.shape[0], dtype=np.int))

batch_size = 64
train_data = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_score, train_loss, test_score, test_loss = [], [], [], []
model = make_model()
loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
for epoch in range(20):
  train_batch_score = []
  test_batch_score = []
  train_batch_loss = []
  test_batch_loss = []
  for batch in train_data:
    model.train()
    optim.zero_grad()
    scores = model(batch[0].to(device=device))
    current_loss = loss(scores, batch[1].to(device=device))
    current_loss.backward()
    optim.step()
    predictions = scores.cpu().detach().numpy().argmax(axis=1)
    train_batch_score.append((predictions == batch[1].numpy()).mean())
    train_batch_loss.append(current_loss.item())
  for batch in test_data:
    model.eval()
    scores = model(batch[0].to(device=device))
    current_loss = loss(scores, batch[1].to(device=device))
    predictions = scores.cpu().detach().numpy().argmax(axis=1)
    test_batch_score.append((predictions == batch[1].numpy()).mean())
    test_batch_loss.append(current_loss.item())
  cetrs = np.mean(train_batch_score)
  cetrl = np.mean(train_batch_loss)
  cetts = np.mean(test_batch_score)
  cettl = np.mean(test_batch_loss) 
  
  print(epoch, ' ', 'train_score = ', cetrs, " test_score = ", cetts, " train_loss = ", cetrl, " test_loss = ", cettl)

0   train_score =  0.275 test_score =  0.27777777777777773 train_loss =  1.5091567249478046 test_loss =  1.3413664247564991
1   train_score =  0.2625 test_score =  0.28958333333333336 train_loss =  1.3542324957755079 test_loss =  1.2862368074570718
2   train_score =  0.5669642857142857 test_score =  0.6284722222222222 train_loss =  1.1790059991515756 test_loss =  0.9930268297989214
3   train_score =  0.6339285714285714 test_score =  0.7548611111111111 train_loss =  0.8810317410817989 test_loss =  0.7508161635741551
4   train_score =  0.6857142857142857 test_score =  0.6909722222222222 train_loss =  0.6664709847542687 test_loss =  0.5781641337848592
5   train_score =  0.7410714285714286 test_score =  0.7458333333333332 train_loss =  0.5417597125240943 test_loss =  0.46035008379570524
6   train_score =  0.8178571428571428 test_score =  0.8618055555555556 train_loss =  0.4142036550041839 test_loss =  0.3273434925648821
7   train_score =  0.85625 test_score =  0.6673611111111111 train_loss